# MusaDAC

# Libararies Installation
PIP3 FOR **MACOS** and PIP FOR **UBUNTU**

In [1]:
!pip3 install d2l
!pip3 install pillow
!pip3 install pytesseract
!pip3 install transformers
!pip3 install timm

# Libraries Import

In [1]:
import torch
from torch import nn
#from d2l import torch as d2l
import pandas as pd
from tqdm import tqdm
from PIL import Image
import pytesseract
import cv2
from sklearn import preprocessing
import os
import numpy as np
import warnings
import os
from torchvision.io import read_image
from torch.utils.data import Dataset
import torch.nn.functional as F
warnings.filterwarnings('ignore')

# INTIAL PARAMETERS TO USE
These Parameter are for optimizing and enabling different features in the notebook such as training loading or using pretrained models.

In [2]:
train_bert = False
device = "cuda" if torch.cuda.is_available() else "cpu"

# PRE TRAINED BERT MODEL FOR WORD EMBEDDINGS


### [Pretrained Model BERT](https://d2l.ai/chapter_natural-language-processing-pretraining/bert-pretraining.html)

In [3]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased", padding=True)
model_betr = DistilBertModel.from_pretrained("distilbert-base-uncased").to(device)

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model_betr(**inputs.to(device))

last_hidden_states = outputs.last_hidden_state

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
def get_bert_encoding(tokens_a):
    spt = " ". join(str(e) for e in tokens_a)
    inputs = tokenizer(spt, return_tensors="pt")
    inp = inputs['input_ids']
    inp = inp.detach().numpy()[0]
    inp = np.pad(inp, [0, 30], mode='constant')
    inp = inp[0:30]
    inputs['input_ids'] = torch.from_numpy(np.asarray([inp]))
    inp = inputs['attention_mask']
    inp = inp.detach().numpy()[0]
    inp = np.pad(inp, [0, 30], mode='constant', constant_values=1)
    inp = inp[0:30]
    inputs['attention_mask'] = torch.from_numpy(np.asarray([inp]))
    outputs = model_betr(**inputs.to(device))
    last_hidden_states = outputs.last_hidden_state
    return last_hidden_states

In [5]:
tokens_a = ['a', 'crane', 'is','ss',"ososS","sss"]
encoded_text = get_bert_encoding(tokens_a)
encoded_text.shape

torch.Size([1, 30, 768])

# PRE TRAINED IMAGE EMBEDDINGS FACEBOOK RESNET50

In [6]:
from transformers import DetrFeatureExtractor, DetrForObjectDetection
from PIL import Image
import requests

In [7]:
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = DetrFeatureExtractor.from_pretrained('facebook/detr-resnet-50')
model_resnet = DetrForObjectDetection.from_pretrained('facebook/detr-resnet-50').to(device)

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model_resnet(**inputs.to(device))

# model predicts bounding boxes and corresponding COCO classes
logits = outputs.logits
bboxes = outputs.pred_boxes

In [8]:
logits.shape

torch.Size([1, 100, 92])

# Reading Meme Labels to Pass In BERT
Meme provided will be passed to the BERT to convert into Word Embeddings and will processed Further 

In [9]:
l_df = pd.read_csv('labels.csv').dropna()

In [10]:
alllabels = list(l_df['text_corrected'])

# Image Processing For Model Training and Testing

Standardizing the Images Sizes and Dimension and Other Parameters for Meeting Basic Neural Network Requirements

Dataset Provided is **memotion_dataset_7k**

In [11]:
from PIL import Image

In [12]:
images_ = list(l_df['image_name'])

In [13]:
""" Images are Converted in to 150 x 150 and Converting to 3 Channel Image RGB """
if(os.path.isdir('proc_images') == False):
    os.mkdir('proc_images')
    image_not = []
    for i in tqdm(range(len(images_))):
        try:
            image = Image.open("images/"+images_[i])
            image = image.resize((82,82),Image.ANTIALIAS)
            image = image.convert('RGB')
            image.save("proc_images/"+images_[i].split('.')[0]+".png", format="png")
        except:
            image_not.append(i)
    for i in image_not:
        l_df = l_df[l_df['image_name'] != i]
else:
    print("Folder Already Exists")

Folder Already Exists


# Image Processing CSV For DataLoader In Pytorch

Generate CSV On Desired Columns and Divide Data on Train (60%) , Test (20%) and Validation (20%)

In [14]:
l_d = l_df[['image_name','overall_sentiment','humour','sarcasm','offensive','motivational','text_corrected']]

In [15]:
for i in l_d.columns:
    print(i,len(l_d[i].unique()))

image_name 6830
overall_sentiment 5
humour 4
sarcasm 4
offensive 4
motivational 2
text_corrected 6784


In [16]:
l_d['text_corrected'].str.split().str.len().max(),l_d['text_corrected'].str.split().str.len().sum()/len(l_d['text_corrected'])

(189, 14.772620790629576)

In [17]:
l_d['overall_sentiment'] = l_d['overall_sentiment'].replace("very_positive", 1)
l_d['overall_sentiment'] = l_d['overall_sentiment'].replace("positive", 1)
l_d['overall_sentiment'] = l_d['overall_sentiment'].replace("very_negative", 2)
l_d['overall_sentiment'] = l_d['overall_sentiment'].replace("negative", 2)
l_d['overall_sentiment'] = l_d['overall_sentiment'].replace("neutral", 0)

In [18]:
l_d['sarcasm'].unique()

array(['general', 'not_sarcastic', 'twisted_meaning', 'very_twisted'],
      dtype=object)

In [19]:
l_d['humour'] = l_d['humour'].replace("not_funny", 0)
l_d['humour'] = l_d['humour'].replace("very_funny", 1)
l_d['humour'] = l_d['humour'].replace("hilarious", 1)
l_d['humour'] = l_d['humour'].replace("funny", 1)

In [20]:
l_d['sarcasm'].unique()

array(['general', 'not_sarcastic', 'twisted_meaning', 'very_twisted'],
      dtype=object)

In [21]:
l_d['sarcasm'] = l_d['sarcasm'].replace("not_sarcastic", 0)
l_d['sarcasm'] = l_d['sarcasm'].replace("general", 1)
l_d['sarcasm'] = l_d['sarcasm'].replace("twisted_meaning", 1)
l_d['sarcasm'] = l_d['sarcasm'].replace("very_twisted", 1)

In [22]:
l_d['offensive'].unique()

array(['not_offensive', 'very_offensive', 'slight', 'hateful_offensive'],
      dtype=object)

In [23]:
l_d['offensive'] = l_d['offensive'].replace("not_offensive", 0)
l_d['offensive'] = l_d['offensive'].replace("very_offensive", 1)
l_d['offensive'] = l_d['offensive'].replace("slight", 1)
l_d['offensive'] = l_d['offensive'].replace("hateful_offensive", 1)

In [24]:
l_d['motivational'].unique()

array(['not_motivational', 'motivational'], dtype=object)

In [25]:
l_d['motivational'] = l_d['motivational'].replace("not_motivational", 0)
l_d['motivational'] = l_d['motivational'].replace("motivational", 1)

In [26]:
l_d['image_name'] = [i.split(".")[0]+'.png' for i in l_d['image_name'] ]

In [27]:
train, validate, test = np.split(l_d.sample(frac=1, random_state=42), [int(.6*len(l_d)), int(.8*len(l_d))])

In [28]:
train.to_csv("train.csv",index=False)
validate.to_csv("validate.csv",index=False)
test.to_csv("test.csv",index=False)

# Data Set Loader in Torch to Feed Neural Network

In [29]:
""" 
Custom Dataset Loader for Pytorch

Batch size taken is 128

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
"""
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0].replace(' ',""))
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        label2 = self.img_labels.iloc[idx, 2]
        label3 = self.img_labels.iloc[idx, 3]
        label4 = self.img_labels.iloc[idx, 4]
        label5 = self.img_labels.iloc[idx, 5]
        text = self.img_labels.iloc[idx, 6]
        text = text.split()
        inputs = feature_extractor(images=image, return_tensors="pt")
        outputs = model_resnet(**inputs.to(device))
        encoded_text = get_bert_encoding(text)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
       
        labelarray = torch.as_tensor(label)
        labelarray2 = torch.as_tensor(label2)
        labelarray3 = torch.as_tensor(label3)
        labelarray4 = torch.as_tensor(label4)
        labelarray5 = torch.as_tensor(label5)
        return image.float(), labelarray, labelarray2, labelarray3, labelarray4, labelarray5, encoded_text
        

In [30]:
train_data = CustomImageDataset(annotations_file="train.csv",
                   img_dir="./proc_images")
val_data = CustomImageDataset(annotations_file="validate.csv",
                   img_dir="./proc_images")
test_data = CustomImageDataset(annotations_file="test.csv",
                   img_dir="./proc_images")

In [31]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=32)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=32)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32)

In [32]:
val_data.__getitem__(19)[1]

tensor(1)

# Image Neural Network
Takes an Input of 20172 Shape Flatten image and Classify 5 Labels

In [33]:
class FirstModel(nn.Module):
    def __init__(self):
        super(FirstModel, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(20172, 1000),
            nn.ReLU(),
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Linear(512, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = F.log_softmax(self.linear_relu_stack(x))
        return logits

In [34]:
class SecondModel(nn.Module):
    def __init__(self):
        super(SecondModel, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(20172, 1000),
            nn.Softmax(),
            nn.Linear(1000, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [35]:
class ThirdModel(nn.Module):
    def __init__(self):
        super(ThirdModel, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(20172, 5096),
            nn.ReLU(),
            nn.Linear(5096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [36]:
class Combined_model(nn.Module):
    def __init__(self, modelA, modelB, modelC):
        super(Combined_model, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.modelC = modelC
        self.classifier = nn.Linear(6, 3)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3, 1024),
            nn.ReLU(),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 3),
        )
        
    def forward(self, x1, x2):
        x11 = self.modelC(x1)
        x2 = self.modelB(x2)
        xs = torch.cat((x11, x2), dim=1)
        xs = self.classifier(xs)
        x3 = self.modelA(x1)
        x = torch.cat((xs, x3), dim=1)
        x = self.classifier(x)

        logits = F.log_softmax(self.linear_relu_stack(x))
        return logits

In [37]:
M1=FirstModel().to(device)
M2=SecondModel().to(device)
M3=ThirdModel().to(device)
model = Combined_model(M1, M2, M3).to(device)

# Text Neural Network

In [38]:
class TextModel(nn.Module): #For images
    def __init__(self):
        super(TextModel, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(23040, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = F.log_softmax(self.linear_relu_stack(x))
        return logits

In [39]:
text_model = TextModel().to(device)

# Combining Image and Text Neural Network
Image Net and Text Net Combined and then Passed throw ANN of 3

In [40]:
class Combination_model(nn.Module):
    def __init__(self, modelA, modelB):
        super(Combination_model, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.classifier = nn.Linear(6, 3)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3, 1024),
            nn.ReLU(),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 24),
        )
        
        self.out1 = nn.Linear(24, 3)
        self.out2 = nn.Linear(24, 2)
        self.out3 = nn.Linear(24, 2)
        self.out4 = nn.Linear(24, 2)
        self.out5 = nn.Linear(24, 2)
        
    def forward(self, x1, x2):
        x1 = self.modelA(x1,x1)
        x2 = self.modelB(x2)
        x = torch.cat((x1, x2), dim=1)
        x = self.classifier(x)

        logits = F.log_softmax(self.linear_relu_stack(x))
        
        out1 = self.out1(logits)
        out2 = self.out2(logits)
        out3 = self.out3(logits)
        out4 = self.out4(logits)
        out5 = self.out5(logits)
        
        return out1, out2, out3, out4, out5

In [41]:
join_model = Combination_model(model, text_model)
join_model.to(device)

Combination_model(
  (modelA): Combined_model(
    (modelA): FirstModel(
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (linear_relu_stack): Sequential(
        (0): Linear(in_features=20172, out_features=1000, bias=True)
        (1): ReLU()
        (2): Linear(in_features=1000, out_features=512, bias=True)
        (3): ReLU()
        (4): Linear(in_features=512, out_features=3, bias=True)
      )
    )
    (modelB): SecondModel(
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (linear_relu_stack): Sequential(
        (0): Linear(in_features=20172, out_features=1000, bias=True)
        (1): Softmax(dim=None)
        (2): Linear(in_features=1000, out_features=3, bias=True)
      )
    )
    (modelC): ThirdModel(
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (linear_relu_stack): Sequential(
        (0): Linear(in_features=20172, out_features=5096, bias=True)
        (1): ReLU()
        (2): Linear(in_features=5096, out_features=2048, bias=True)
        (3): ReLU

In [42]:
learning_rate = 1e-3
batch_size = 32
epochs = 1
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y, y2, y3, y4, y5, z) in enumerate(dataloader):
        pred1, pred2, pred3 ,pred4 ,pred5 = join_model(X.to(device),z.to(device))
        loss = loss_fn(pred1.to(device), y.to(device))
        loss += loss_fn(pred2.to(device), y2.to(device))
        loss += loss_fn(pred3.to(device), y3.to(device))
        loss += loss_fn(pred4.to(device), y4.to(device))
        loss += loss_fn(pred5.to(device), y5.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 2 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    correct2 = 0
    correct3 = 0
    correct4 = 0
    correct5 = 0
    with torch.no_grad():
        for X, y, y2, y3, y4, y5, z in dataloader:
            pred, pred2, pred3, pred4, pred5 = join_model(X.to(device),z.to(device))
            correct += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()
            correct2 += (pred2.argmax(1) == y2.to(device)).type(torch.float).sum().item()
            correct3 += (pred3.argmax(1) == y3.to(device)).type(torch.float).sum().item()
            correct4 += (pred4.argmax(1) == y4.to(device)).type(torch.float).sum().item()
            correct5 += (pred5.argmax(1) == y5.to(device)).type(torch.float).sum().item()
    correct /= size
    correct2 /= size
    correct3 /= size
    correct4 /= size
    correct5 /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, For overall_sentiment")
    print(f"Test Error: \n Accuracy: {(100*correct2):>0.1f}%, For humour")
    print(f"Test Error: \n Accuracy: {(100*correct3):>0.1f}%, For sarcasm")
    print(f"Test Error: \n Accuracy: {(100*correct4):>0.1f}%, For offensive")
    print(f"Test Error: \n Accuracy: {(100*correct5):>0.1f}%, For motivational")

# Training and Testing Joined Model

In [43]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(join_model.parameters(), lr=learning_rate)

epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, join_model, loss_fn, optimizer)
    test_loop(val_dataloader, join_model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 5.084747  [    0/ 4098]
loss: 54.339207  [   64/ 4098]
loss: 53.433208  [  128/ 4098]
loss: 5.979975  [  192/ 4098]
loss: 5.524271  [  256/ 4098]
loss: 6.125224  [  320/ 4098]
loss: 6.288033  [  384/ 4098]
loss: 3.679430  [  448/ 4098]
loss: 5.052067  [  512/ 4098]
loss: 5.723091  [  576/ 4098]
loss: 13.444696  [  640/ 4098]
loss: 4.266817  [  704/ 4098]
loss: 4.084516  [  768/ 4098]
loss: 3.690948  [  832/ 4098]
loss: 3.907033  [  896/ 4098]
loss: 3.609365  [  960/ 4098]
loss: 3.621816  [ 1024/ 4098]
loss: 3.350447  [ 1088/ 4098]
loss: 3.548254  [ 1152/ 4098]
loss: 3.080630  [ 1216/ 4098]
loss: 3.563564  [ 1280/ 4098]
loss: 3.270173  [ 1344/ 4098]
loss: 3.287387  [ 1408/ 4098]
loss: 3.511893  [ 1472/ 4098]
loss: 3.599060  [ 1536/ 4098]
loss: 3.265493  [ 1600/ 4098]
loss: 3.588131  [ 1664/ 4098]
loss: 3.558917  [ 1728/ 4098]
loss: 3.328246  [ 1792/ 4098]
loss: 3.224560  [ 1856/ 4098]
loss: 3.671576  [ 1920/ 4098]
loss: 3.330847  [ 1984/ 409

In [44]:
#torch.save(text_model.state_dict(), "text.pt")

In [45]:
#torch.save(model.state_dict(), "img.pt")

In [46]:
torch.save(join_model.state_dict(), "joinedmodel.pt") 

# Image to Text OCR
Convert Image into Text using OCR [pytesseract](https://pyimagesearch.com/2017/07/10/using-tesseract-ocr-python/)

In [ ]:
text = pytesseract.image_to_string(Image.open("images/image_3.jpg"))
print(text)